In [97]:
from pyforest import *
from sklearn.preprocessing import Binarizer, binarize, KBinsDiscretizer

In [98]:
num_list = [
    [-1000, 0],
    [500, -3000],
    [100, 650]
]

In [99]:
binarizer = Binarizer(threshold=[0, 100])

In [100]:
binarizer.fit(num_list)

Binarizer(copy=True, threshold=[0, 100])

In [101]:
binarized_list = binarizer.transform(num_list)

binarized_list

array([[0, 0],
       [1, 0],
       [1, 1]])

In [102]:
diet_data = pd.read_csv('diet_data.csv')

In [103]:
diet_data.head()

,Date,Stone,Pounds,Ounces,weight_oz,calories,cals_per_oz,five_donuts,walk,run,wine,prot,weight,change
0,7/30/2018,12.0,2.0,6.0,2726.0,1950.0,0.72,1.0,1.0,0.0,0.0,0.0,0.0,-30.0
1,7/31/2018,12.0,0.0,8.0,2696.0,2600.0,0.96,1.0,0.0,0.0,0.0,0.0,0.0,8.0
2,8/1/2018,12.0,1.0,0.0,2704.0,2500.0,0.92,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,8/2/2018,12.0,1.0,0.0,2704.0,1850.0,0.68,1.0,1.0,0.0,1.0,0.0,0.0,-40.0
4,8/3/2018,11.0,12.0,8.0,2664.0,2900.0,1.09,1.0,1.0,0.0,0.0,0.0,0.0,14.0


In [104]:
diet_data = diet_data.dropna()

diet_data = diet_data.drop(['Date', 'Stone', 'Pounds', 'Ounces'], axis = 1)

In [105]:
diet_data = diet_data.astype('float64')

In [106]:
diet_data.head()

,weight_oz,calories,cals_per_oz,five_donuts,walk,run,wine,prot,weight,change
0,2726.0,1950.0,0.72,1.0,1.0,0.0,0.0,0.0,0.0,-30.0
1,2696.0,2600.0,0.96,1.0,0.0,0.0,0.0,0.0,0.0,8.0
2,2704.0,2500.0,0.92,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2704.0,1850.0,0.68,1.0,1.0,0.0,1.0,0.0,0.0,-40.0
4,2664.0,2900.0,1.09,1.0,1.0,0.0,0.0,0.0,0.0,14.0


In [107]:
diet_data.calories.median()

2575.0

In [108]:
binarizer = Binarizer(threshold=2575)
diet_data['calories_above_median'] = binarizer.fit_transform(diet_data[['calories']])

In [109]:
diet_data.head()

,weight_oz,calories,cals_per_oz,five_donuts,walk,run,wine,prot,weight,change,calories_above_median
0,2726.0,1950.0,0.72,1.0,1.0,0.0,0.0,0.0,0.0,-30.0,0.0
1,2696.0,2600.0,0.96,1.0,0.0,0.0,0.0,0.0,0.0,8.0,1.0
2,2704.0,2500.0,0.92,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2704.0,1850.0,0.68,1.0,1.0,0.0,1.0,0.0,0.0,-40.0,0.0
4,2664.0,2900.0,1.09,1.0,1.0,0.0,0.0,0.0,0.0,14.0,1.0


In [110]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

In [132]:
x_train, x_test, y_train, y_test = train_test_split(diet_data.drop(['walk'], axis=1), diet_data['walk'])

In [133]:
model = GradientBoostingClassifier().fit(x_train, y_train)

In [134]:
model.score(x_test, y_test)

0.6857142857142857

In [135]:
k_bins = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')

In [136]:
k_bins_array = k_bins.fit_transform(diet_data[['calories']])

In [137]:
k_bins_array.shape

(140, 1)

In [141]:
k_bins_df = pd.DataFrame(data = k_bins_array,
                        columns = ['BinnedCalories'])

k_bins_df.shape

(140, 1)

In [139]:
k_bins_df.BinnedCalories.value_counts()

0.0    105
1.0     32
2.0      3
Name: BinnedCalories, dtype: int64

In [140]:
diet_data_k = pd.concat([diet_data, k_bins_df], axis=1)
diet_data_k

,weight_oz,calories,cals_per_oz,five_donuts,walk,run,wine,prot,weight,change,calories_above_median,BinnedCalories
0,2726.0,1950.0,0.72,1.0,1.0,0.0,0.0,0.0,0.0,-30.0,0.0,0.0
1,2696.0,2600.0,0.96,1.0,0.0,0.0,0.0,0.0,0.0,8.0,1.0,0.0
2,2704.0,2500.0,0.92,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2704.0,1850.0,0.68,1.0,1.0,0.0,1.0,0.0,0.0,-40.0,0.0,0.0
4,2664.0,2900.0,1.09,1.0,1.0,0.0,0.0,0.0,0.0,14.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
138,2640.0,5750.0,2.18,0.0,0.0,0.0,0.0,0.0,0.0,16.0,1.0,0.0
139,2656.0,2950.0,1.11,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
140,2656.0,1950.0,0.73,1.0,1.0,0.0,0.0,0.0,0.0,-24.0,0.0,NaN
141,2632.0,1900.0,0.72,0.0,1.0,0.0,0.0,0.0,0.0,-4.0,0.0,NaN


In [142]:
calories_edges = k_bins.bin_edges_

In [144]:
calories_edges[0]

array([1400.        , 3983.33333333, 6566.66666667, 9150.        ])